In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import KFold

In [3]:
train_dataset = pd.read_csv("/Users/danielguarnizo/Desktop/HACK4SDS/Dataset_DAY1/Data/train_set.csv", delimiter=';')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/danielguarnizo/Desktop/HACK4SDS/Dataset_DAY1/Data/train_set.csv'

In [ ]:
train_dataset.head(30)

,application_ID,decision_date,company_ID,external_score_ver01,external_score_ver02,late_payment_score,external_score_late_payment_integrated,external_score_moderate,external_score_adverse,external_score_ver03,...,avg_count_enti_affidanti,std_count_enti_affidanti,max_count_enti_affidanti,last_count_enti_affidanti,avg_count_numero_prima_info,std_count_numero_prima_info,max_count_numero_prima_info,last_count_numero_prima_info,days_to_default,target
0,a1Q7R00000ej2yjUAA,2021-11-30,7256587870,10,3,NaN,NaN,NaN,NaN,MISSING,...,1,0,1,1,0,0,0,0,522,1
1,a1Q2X00000ZWC5LUAX,2020-10-06,6178307100,7,3,NaN,NaN,NaN,NaN,H,...,1,0,1,1,2,0,2,2,1498,0
2,a1Q2X00000XcCCQUA3,2020-02-11,7692855390,7,3,NaN,NaN,NaN,NaN,MISSING,...,1,0,1,1,1,0,1,1,779,1
3,a1Q7R00000ejSs3UAE,2022-01-18,5752241730,8,2,NaN,NaN,NaN,NaN,MISSING,...,1,0,1,1,"0,5","0,522232967867094",1,0,1498,0
4,a1Q7R00000eiRidUAE,2021-09-16,7533506540,4,1,NaN,NaN,NaN,NaN,MISSING,...,0,0,0,0,0,0,0,0,1498,0
5,a1Q7R00000c7W0XUAU,2022-05-12,8474427550,6,1,NaN,NaN,NaN,NaN,C,...,"2,25","0,452267016866645",3,3,"2,91666666666667","0,668557923421522",4,1,1498,0
6,a1Q7R00000ZWOTeUAP,2020-12-17,7914332300,7,3,NaN,NaN,NaN,NaN,E,...,0,0,0,0,0,0,0,0,1498,0
7,a1Q7R00000ZWI9YUAX,2020-11-05,3216373770,6,1,NaN,NaN,NaN,NaN,MISSING,...,6,0,6,6,"0,25","0,621581560508061",2,2,1007,1
8,a1Q7R00000a2vntUAA,2021-04-14,4534845540,10,3,NaN,NaN,NaN,NaN,MISSING,...,1,0,1,1,1,0,1,1,752,1
9,a1Q7R00000a2sxMUAQ,2021-04-13,5199648490,7,2,NaN,NaN,NaN,NaN,G,...,0,0,0,0,0,0,0,0,1498,0


In [ ]:
## drop features
def Drop_unneed_columns(test, dataset):
    cols= ['application_ID', 'company_ID']
    dataset= dataset.drop(columns=cols)
    return dataset

In [ ]:
def Nan_values(dataset):
    column_names = dataset.columns.tolist()
    drop_columns = []
    for name in column_names:
        nan_count = dataset[name].isna().sum()
        print(f"column {name}: {nan_count}")
        if (nan_count/28000) > 0.5:
            print(f"Number of NaN values in column '{name}': {nan_count}")
            drop_columns.append(name)
    return drop_columns

In [ ]:
def Replace_cate_to_value(column_name, dataset):
    # Extract categories

    # Extract unique category names from the column
    unique_categories = dataset[column_name].unique()

    # convert 'numpy.ndarray' in to a python list
    l = unique_categories.tolist()
    
    if 'MISSING' in l:
        l.remove('MISSING')
        l.sort(reverse=True)

    print(l)
    
    dic = { l[i]:i+1 for i in range(0, len(l))}

    # Replace values in the column based on the dictionary mapping
    dataset[column_name] = dataset[column_name].replace(dic)
    return dic, dataset

In [ ]:
def Category_values(dataset):
    column_names = ['industry_sector', 'decision_date' ,'region', 'geo_area','external_score_ver03', 'province','juridical_form']
    dic = {}
    for column_name in column_names:
        category_dic, dataset = Replace_cate_to_value(column_name, dataset)
        dic[column_name] = category_dic
    return dic, dataset

In [ ]:
def Replace_bool_toNumbers(dataset):
    dataset['cr_available'] = [int(dataset['cr_available'][i]) for i in range(len(dataset['cr_available']))]
    dataset['cr_available']
    return dataset

In [ ]:
def mean_var03(dataset):
    s0, s1, c0, c1 = 0,0,0,0
    # unique_labels = dataset['target'].unique()
    for index, row in dataset.iterrows():
        if row['external_score_ver03'] != 'MISSING':
            if row['target'] == 0:
                s0 += row['external_score_ver03']
                c0 +=1
            elif row['target'] == 1:
                s1 +=  row['external_score_ver03']
                c1 += 1

    m0 = round(s0/c0)
    m1 = round(s1/c1)
    print(m0)
    print(m1)
    return m0,m1

In [ ]:
def Replace_missing(dataset, m0, m1):
    # Assuming df is your DataFrame and 'column_to_change' is the column you want to change
    # 'condition_column' is the column based on which you want to change the content
    dataset.loc[(dataset['target'] == 1) & (dataset['external_score_ver03'] == 'MISSING'), 'external_score_ver03'] = m1
    dataset.loc[(dataset['target'] == 0) & (dataset['external_score_ver03'] == 'MISSING'), 'external_score_ver03'] = m0
    dataset['external_score_ver03']

    # For example, if you want to change the content of 'column_to_change' to 'new_value' where 'condition_column' is True
    # Replace 'new_value', 'column_to_change', and 'condition_column' with your actual values
    return dataset

In [ ]:
# Drop columns 
train_dataset = Drop_unneed_columns(False,train_dataset)
drop_columns = Nan_values(train_dataset)
train_dataset = train_dataset.drop(columns=drop_columns)

column decision_date: 0
column external_score_ver01: 0
column external_score_ver02: 0
column late_payment_score: 27488
Number of NaN values in column 'late_payment_score': 27488
column external_score_late_payment_integrated: 27488
Number of NaN values in column 'external_score_late_payment_integrated': 27488
column external_score_moderate: 27208
Number of NaN values in column 'external_score_moderate': 27208
column external_score_adverse: 27208
Number of NaN values in column 'external_score_adverse': 27208
column external_score_ver03: 0
column age: 0
column province: 2654
column juridical_form: 0
column industry_sector: 0
column gross_margin_ratio: 0
column core_income_ratio: 0
column cash_asset_ratio: 0
column consolidated_liabilities_ratio: 0
column tangible_assets_ratio: 0
column revenues: 0
column cr_available: 0
column region: 0
column geo_area: 0
column last_statement_age: 0
column overrun_freq_a_revoca_autoliquidanti: 0
column avg_tension_a_revoca_autoliquidanti: 0
column std_te

In [ ]:
# replace bool values to numerical ones 
category_dics, train_dataset = Category_values(train_dataset)
train_dataset = Replace_bool_toNumbers(train_dataset)

['Servizi', 'Costruzioni e materiali per costruzioni', 'Distribuzione', 'Trasporti', 'Utility', 'Editoria e stampa', 'Elettrotecnica ed elettronica', 'Altri beni di consumo', 'Sistema moda', 'Metallurgia e prodotti in metallo', 'Meccanica', 'Chimica di base e intermedi', 'Elettrodomestici', 'Agricoltura', 'Alimentare', 'Energia ed estrazione', 'Largo consumo / attività ricreativo-culturali', 'Mezzi di trasporto', 'Holding, finanziarie ed altro', 'Farmaceutica']
['2021-11-30', '2020-10-06', '2020-02-11', '2022-01-18', '2021-09-16', '2022-05-12', '2020-12-17', '2020-11-05', '2021-04-14', '2021-04-13', '2021-09-23', '2021-06-16', '2020-07-24', '2022-02-16', '2021-11-10', '2020-10-16', '2021-04-27', '2021-03-30', '2021-06-14', '2021-11-22', '2021-07-05', '2022-01-27', '2021-02-18', '2021-02-17', '2022-01-24', '2021-10-23', '2021-09-14', '2022-04-27', '2022-03-18', '2021-10-28', '2022-01-28', '2021-10-08', '2022-01-14', '2020-10-02', '2021-03-15', '2021-04-29', '2022-05-03', '2022-04-05', '

In [ ]:
# v03 column with missing values 
m0, m1= mean_var03(train_dataset)
train_dataset = Replace_missing(train_dataset, m0, m1)

8
10


In [ ]:
def normalized_data(dataset):
    # Replace commas with periods in all columns
    dataset = dataset.replace(',', '.', regex=True)
    print(dataset.dtypes)
    dataset = dataset.astype('float32')

    # check if the dataset has any nan value
    has_nan_values = dataset.isna().any().any()

    if has_nan_values:
        print("DataFrame contains NaN values.")
    else:
        print("DataFrame does not contain any NaN values.")

    return dataset

In [ ]:
train_dataset = normalized_data(train_dataset)
train_dataset.head(30)

decision_date                             int64
external_score_ver01                      int64
external_score_ver02                      int64
external_score_ver03                      int64
age                                       int64
province                                  int64
juridical_form                            int64
industry_sector                           int64
gross_margin_ratio                       object
core_income_ratio                        object
cash_asset_ratio                         object
consolidated_liabilities_ratio           object
tangible_assets_ratio                    object
revenues                                 object
cr_available                              int64
region                                    int64
geo_area                                  int64
last_statement_age                        int64
overrun_freq_a_revoca_autoliquidanti     object
avg_tension_a_revoca_autoliquidanti      object
std_tension_a_revoca_autoliquidanti     

,decision_date,external_score_ver01,external_score_ver02,external_score_ver03,age,province,juridical_form,industry_sector,gross_margin_ratio,core_income_ratio,...,avg_count_enti_affidanti,std_count_enti_affidanti,max_count_enti_affidanti,last_count_enti_affidanti,avg_count_numero_prima_info,std_count_numero_prima_info,max_count_numero_prima_info,last_count_numero_prima_info,days_to_default,target
0,1.0,10.0,3.0,10.0,15.0,1.0,1.0,1.0,0.464637,0.012593,...,1.000000,0.000000,1.0,1.0,0.000000,0.000000,0.0,0.0,522.0,1.0
1,2.0,7.0,3.0,7.0,5.0,2.0,2.0,2.0,0.372340,0.115385,...,1.000000,0.000000,1.0,1.0,2.000000,0.000000,2.0,2.0,1498.0,0.0
2,3.0,7.0,3.0,10.0,5.0,3.0,1.0,3.0,0.270000,0.006369,...,1.000000,0.000000,1.0,1.0,1.000000,0.000000,1.0,1.0,779.0,1.0
3,4.0,8.0,2.0,8.0,6.0,4.0,1.0,1.0,0.419929,0.152174,...,1.000000,0.000000,1.0,1.0,0.500000,0.522233,1.0,0.0,1498.0,0.0
4,5.0,4.0,1.0,8.0,5.0,5.0,1.0,1.0,0.526316,0.083333,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1498.0,0.0
5,6.0,6.0,1.0,12.0,2.0,6.0,1.0,4.0,0.673177,0.026609,...,2.250000,0.452267,3.0,3.0,2.916667,0.668558,4.0,1.0,1498.0,0.0
6,7.0,7.0,3.0,10.0,3.0,7.0,2.0,1.0,1.666667,0.017544,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1498.0,0.0
7,8.0,6.0,1.0,10.0,9.0,8.0,1.0,5.0,0.307391,0.010648,...,6.000000,0.000000,6.0,6.0,0.250000,0.621582,2.0,2.0,1007.0,1.0
8,9.0,10.0,3.0,10.0,2.0,9.0,1.0,2.0,0.457291,0.068892,...,1.000000,0.000000,1.0,1.0,1.000000,0.000000,1.0,1.0,752.0,1.0
9,10.0,7.0,2.0,8.0,12.0,10.0,3.0,6.0,0.221461,-0.045290,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1498.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
y = train_dataset['days_to_default'].to_numpy() # labels
X = train_dataset.drop(columns=['days_to_default']).to_numpy()